## Imports

In [66]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict
from typing import Union, Tuple, List
import numpy as np

from scipy import ndimage as ndi
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.utils import hole_filling

from skimage import filters
from skimage.segmentation import *
from skimage.morphology import remove_small_holes   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from skimage.morphology import disk, binary_dilation, binary_erosion, erosion, dilation, ball, closing, binary_closing

from infer_subc.core.file_io import (read_tiff_image,
                                     read_czi_image,
                                     read_ome_image,
                                     import_inferred_organelle,
                                     export_inferred_organelle,
                                     list_image_files)

                                             
from infer_subc.core.img import *
from infer_subc.organelles import (get_nuclei, 
                                   non_linear_cellmask_transform,
                                   choose_max_label_cellmask_union_nucleus)
from infer_subc.constants import (TEST_IMG_N,
                                  NUC_CH ,
                                  LYSO_CH ,
                                  MITO_CH ,
                                  GOLGI_CH ,
                                  PEROX_CH ,
                                  ER_CH ,
                                  LD_CH ,
                                  RESIDUAL_CH) 

LD_CH = 6
NUC_CH = 7
LYSO_CH = 4
MITO_CH = 3
GOLGI_CH = 2
PEROX_CH = 1
ER_CH = 0
PM_CH = 5
RESIDUAL_CH = 8
TEST_IMG_N = 16


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
test_viewer = napari.Viewer()

01-Mar-24 15:17:44 - vispy    - WARNING  - QWindowsWindow::setGeometry: Unable to set geometry 1280x1208+12+46 (frame: 1302x1264+1+1) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY1". Resulting geometry: 1280x1050+12+46 (frame: 1302x1106+1+1) margins: 11, 45, 11, 11 minimum size: 385x491 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=792,1038 maxtrack=0,0)


01-Mar-24 15:18:47 - vispy    - WARNING  - QWindowsWindow::setGeometry: Unable to set geometry 1920x988+0+34 (frame: 1942x1044-11-11) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY1". Resulting geometry: 1920x986+0+34 (frame: 1942x1042-11-11) margins: 11, 45, 11, 11 minimum size: 538x494 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=1098,1044 maxtrack=0,0)


## Get & Load Image for Processing

In [35]:
test_img_n = TEST_IMG_N

data_root_path = Path(os.path.expanduser("~")) / "Documents/Python Scripts/Infer-subc-2D"

in_data_path = data_root_path / "raw mcz"
im_type = ".tiff"

img_file_list = list_image_files(in_data_path,im_type)
test_img_name = img_file_list[test_img_n]

out_data_path = data_root_path / "segmented"
if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

In [36]:
img_data,meta_dict = read_czi_image(test_img_name) #read_czi_image works for tiff images as well

channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

In [37]:
test_viewer.add_image(img_data)

<Image layer 'img_data' at 0x24b3b382380>

## Membrane

In [38]:
"""
pm_img is the RAW plasma membrane image
pm_obj is the binary object of the plasma membrane
invert_pm_img is the inverted RAW plasma membrane image
invert_pm_obj is the binary object of the inverted plasma membrane
"""

###################
# INPUT
###################
pm_img = select_channel_from_raw(img_data, PM_CH)


## Nuclei

In [39]:
# Import nuclei segmentation from output folder, or 
# run nuclei segmentation with hard coded segmentation settings (not ideal for most scenarios)
nuclei_labels = get_nuclei(img_data, meta_dict, out_data_path)

loaded  inferred 3D `nuclei`  from C:\Users\zscoman\Documents\Python Scripts\Infer-subc-2D\segmented 


In [40]:
test_viewer.add_labels(nuclei_labels)

<Labels layer 'nuclei_labels' at 0x24b1b0fe110>

## Define _rescale_intensity

In [41]:
def _rescale_intensity(in_img: np.ndarray):
    #rescales the intensity of input image on a scale of 0 to 10
    out_img = ((in_img - in_img.min())/(in_img.max() - in_img.min()))*10
    return out_img

## Composite Images

In [42]:
###################
# CORE_PROCESSING
###################

threshadjust = 2.5
pm_image, d = log_transform(pm_img.copy())
threshold = threshold_otsu_log(pm_image) 
pm_obj = (pm_img > (inverse_log_transform(threshold, d) * threshadjust))

invert_pm_obj = np.invert(pm_obj)

invert_pm_img = abs(np.max(img_data[PM_CH]) - img_data[PM_CH]) 

#################
# Hole Filling
#################
minhole = 0
maxhole = 100000

#invert_pm_obj = hole_filling(invert_pm_obj, hole_min=minhole, hole_max=maxhole, fill_2d=True)

In [43]:
test_viewer.add_image(invert_pm_obj)

<Image layer 'invert_pm_obj' at 0x24bd361cd90>

In [44]:
#
composite_img_raw1 = (1. * _rescale_intensity(img_data[LYSO_CH].copy().astype(np.double)) +
                      1. * _rescale_intensity(img_data[PEROX_CH].copy().astype(np.double)) + 
                      2. * _rescale_intensity(img_data[ER_CH].copy().astype(np.double)) + 
                      1. * _rescale_intensity(img_data[MITO_CH].copy().astype(np.double)) +
                      0. * _rescale_intensity(img_data[NUC_CH].copy().astype(np.double)) +
                      0. * _rescale_intensity(img_data[PM_CH].copy().astype(np.double)) + 
                      0. * _rescale_intensity(invert_pm_img.copy().astype(np.double)))

#
composite_img_raw2 = (1. * _rescale_intensity(img_data[LYSO_CH].copy().astype(np.double)) +
                      1. * _rescale_intensity(img_data[PEROX_CH].copy().astype(np.double)) + 
                      2. * _rescale_intensity(img_data[ER_CH].copy().astype(np.double)) + 
                      1. * _rescale_intensity(img_data[MITO_CH].copy().astype(np.double)) +
                      0. * _rescale_intensity(img_data[NUC_CH].copy().astype(np.double)) +
                      0. * _rescale_intensity(img_data[PM_CH].copy().astype(np.double)) +
                      1. * _rescale_intensity(invert_pm_img.copy().astype(np.double)))

In [45]:
test_viewer.add_image(composite_img_raw1)
test_viewer.add_image(composite_img_raw2)

<Image layer 'composite_img_raw2' at 0x24bc87a7490>

Need to add something to separate two cells that are close together!!!

## Closing

In [46]:
fp = disk(15)
close = np.zeros_like(composite_img_raw1)
close2 = np.zeros_like(composite_img_raw2)
for i in range(len(composite_img_raw1)):
    close[i] = closing(composite_img_raw1.copy()[i], footprint=fp)
    close2[i] = closing(composite_img_raw2.copy()[i], footprint=fp)

In [47]:
test_viewer.add_image(close)

<Image layer 'close' at 0x24bc86d18a0>

## Thresholding

In [48]:
thresh_method = 'med'
cutoff_size1 =  0
thresh_adj1 = 0.6
cutoff_size2 =  0
thresh_adj2 = 0.7
close_thresh = masked_object_thresh(min_max_intensity_normalization(close.copy()), 
                                    global_method=thresh_method, 
                                    cutoff_size=cutoff_size1, 
                                    local_adjust=thresh_adj1) 
close_thresh2 = masked_object_thresh(min_max_intensity_normalization(close2.copy()),
                                     global_method=thresh_method,
                                     cutoff_size=cutoff_size2,
                                     local_adjust=thresh_adj2) 

In [49]:
test_viewer.add_image(close_thresh2)

<Image layer 'close_thresh2' at 0x24b423225c0>

In [50]:
#Picks the nuclei of the desired cell's label number
keep_nuc = get_max_label((close_thresh), dilation(nuclei_labels))
print(keep_nuc)

1


In [51]:
#Removes all nuclei aside from desired nuclei
single_nuc = np.zeros_like(nuclei_labels)
single_nuc[nuclei_labels == keep_nuc] = 1


In [52]:
#Combines nuclei and close_thresh
combo_thresh = np.zeros_like(close_thresh)
combo_thresh2 = np.zeros_like(close_thresh2)
foot = disk(10)
for i in range(len(composite_img_raw1)):
    combo_thresh[i] = close_thresh[i] + binary_dilation(single_nuc.astype(bool)[i])
    combo_thresh2[i] = close_thresh2[i] + binary_dilation(single_nuc.astype(bool)[i])
#combo_thresh = close_thresh + binary_dilation(single_nuc.astype(bool))
#combo_thresh2 = close_thresh2 + binary_dilation(single_nuc.astype(bool))

In [53]:
test_viewer.add_image(combo_thresh2)

<Image layer 'combo_thresh2' at 0x24b430b5b10>

In [54]:
#Filling remaining holes
hole_min = 0
hole_max = 100
small_object = 0

thresh = fill_and_filter_linear_size(combo_thresh, 
                                      hole_min=hole_min, 
                                      hole_max=hole_max, 
                                      min_size=small_object)
thresh2 = fill_and_filter_linear_size(combo_thresh2, 
                                      hole_min=hole_min, 
                                      hole_max=hole_max, 
                                      min_size=small_object)

In [55]:
test_viewer.add_image(thresh2)

<Image layer 'thresh2' at 0x24b43cce5f0>

## Mask Plasma Membrane Invert

In [56]:
test_viewer.add_image(invert_pm_obj)

<Image layer 'invert_pm_obj [1]' at 0x24b42140af0>

In [57]:
masked_invert_pm = np.zeros_like(invert_pm_obj)
masked_invert_pm[(invert_pm_obj == thresh) & 
                 (invert_pm_obj == 1) & 
                 (thresh == 1)] = 1

In [58]:
test_viewer.add_image(masked_invert_pm)

<Image layer 'masked_invert_pm' at 0x24b4726e530>

In [59]:
ED_masked_invert_pm = dilation(erosion(masked_invert_pm.copy()))

In [60]:
test_viewer.add_image(pm_img)

<Image layer 'pm_img' at 0x24b473d5180>

## Watershed

In [61]:
cellmask_labels = masked_inverted_watershed(close, 
                                            single_nuc, 
                                            thresh,
                                            method='3D')

cellmask_labels2 = masked_inverted_watershed(close2, 
                                             single_nuc, 
                                             thresh2,
                                             method='3D')

In [62]:
cellmask_combo = cellmask_labels.astype(bool) + cellmask_labels2.astype(bool)
hole_min = 0
hole_max = 100000
for n in range(len(cellmask_combo)):
    cellmask_combo[n] = binary_dilation(cellmask_combo.copy()[n], footprint=fp)
cellmask_combo = hole_filling(cellmask_combo.copy(), hole_min=hole_min, hole_max=hole_max, fill_2d=True)
for n in range(len(cellmask_combo)):
    cellmask_combo[n] = binary_erosion(cellmask_combo.copy()[n], footprint=fp)

In [63]:
test_viewer.add_labels(cellmask_combo)

<Labels layer 'cellmask_combo' at 0x24b7fe678b0>

In [65]:
out_file_n = export_inferred_organelle(cellmask_combo, "cell", meta_dict, out_data_path)

saved file: 20240202_iN D7 TRC2 ctrl_Z 15_Linear unmixing_0_cmle.ome-cell
